In [1]:
# This is for CJV

import numpy as np
import pandas as pd
from scipy.stats import rankdata

matrix = np.array([
    [0.156, 0.146, 0.172, 0.140],
    [0.177, 0.101, 0.307, 0.104],
    [0.070, 0.092, 0.058, 0.084],
    [0.024, 0.126, 0.047, 0.128],
    [0.121, 0.184, 0.038, 0.206],
    [0.208, 0.146, 0.152, 0.116],
    [0.025, 0.126, 0.068, 0.091],
    [0.219, 0.079, 0.159, 0.131]
])

sites = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5', 'Site6', 'Site7', 'Site8']
criteria = ['C1', 'C2', 'C3', 'C4']
criteria_types = ['max', 'max', 'max', 'max'] 
weights = np.array([0.25377794, 0.35076977, 0.39287723, 0.00257506])

# NORMALIZATION FUNCTION
def vector_normalize(matrix):
    norm_matrix = np.zeros_like(matrix, dtype=float)
    for j in range(matrix.shape[1]):
        col = matrix[:, j]
        norm = np.sqrt(np.sum(col ** 2))
        norm_matrix[:, j] = col / norm if norm != 0 else col
    return norm_matrix

# MULTIMOORA METHOD
def multimoora(matrix, weights, criteria_types, sites, criteria):
    n, m = matrix.shape

    # 1. Ratio System (vector normalization, then weighted sum)
    norm_matrix = vector_normalize(matrix)
    for j in range(m):
        if criteria_types[j] == 'min':
            norm_matrix[:, j] = -norm_matrix[:, j]
    ratio_scores = np.dot(norm_matrix, weights)

    # 2. Reference Point Approach (use the same normalization)
    ref_point = np.max(norm_matrix, axis=0)
    # For cost criteria, reference point is min (but we already multiplied by -1)
    ref_distances = np.max(np.abs(norm_matrix - ref_point), axis=1)

    # 3. Full Multiplicative Form
    mult_form = np.ones(n)
    for i in range(n):
        prod = 1.0
        for j in range(m):
            val = matrix[i, j]
            if criteria_types[j] == 'max':
                prod *= val ** weights[j]
            else:  # 'min'
                # Avoid division by zero
                prod *= (1.0 / val) ** weights[j] if val != 0 else 0
        mult_form[i] = prod

    # Rankings
    ratio_rank = rankdata(-ratio_scores, method='min')
    ref_rank = rankdata(ref_distances, method='min')
    mult_rank = rankdata(-mult_form, method='min')

    # Aggregate (Borda count: sum of ranks, lower is better)
    total_rank = ratio_rank + ref_rank + mult_rank

    df = pd.DataFrame({
        'Ratio Score': ratio_scores,
        'Ratio Rank': ratio_rank,
        'Reference Distance': ref_distances,
        'Reference Rank': ref_rank,
        'Multiplicative Score': mult_form,
        'Multiplicative Rank': mult_rank,
        'Total Rank': total_rank,
        'Final Rank': rankdata(total_rank, method='min')
    }, index=sites).sort_values('Final Rank')

    return df

# RUN MULTIMOORA
result_df = multimoora(matrix, weights, criteria_types, sites, criteria)
print(result_df)


       Ratio Score  Ratio Rank  Reference Distance  Reference Rank  \
Site2     0.488819           1            0.277414               1   
Site1     0.395619           3            0.314841               2   
Site6     0.409358           2            0.361484               4   
Site8     0.358288           4            0.345159               3   
Site5     0.288139           5            0.627350               8   
Site7     0.199562           6            0.557385               5   
Site3     0.185559           7            0.580707               6   
Site4     0.179960           8            0.606360               7   

       Multiplicative Score  Multiplicative Rank  Total Rank  Final Rank  
Site2              0.180250                    1           3           1  
Site1              0.158333                    3           8           2  
Site6              0.162172                    2           8           2  
Site8              0.134870                    4          11         

In [2]:
# This is for Freight

import numpy as np
import pandas as pd
from scipy.stats import rankdata

matrix = np.array([
    [0.089, 0.106, 0.110, 0.140],
    [0.172, 0.099, 0.291, 0.104],
    [0.206, 0.095, 0.072, 0.084],
    [0.011, 0.132, 0.061, 0.128],
    [0.128, 0.243, 0.052, 0.206],
    [0.178, 0.120, 0.178, 0.116],
    [0.000, 0.132, 0.073, 0.091],
    [0.217, 0.074, 0.163, 0.131]
])

sites = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5', 'Site6', 'Site7', 'Site8']
criteria = ['C1', 'C2', 'C3', 'C4']
criteria_types = ['max', 'max', 'max', 'max'] 
weights = np.array([0.25377794, 0.35076977, 0.39287723, 0.00257506]) 

# NORMALIZATION FUNCTION
def vector_normalize(matrix):
    norm_matrix = np.zeros_like(matrix, dtype=float)
    for j in range(matrix.shape[1]):
        col = matrix[:, j]
        norm = np.sqrt(np.sum(col ** 2))
        norm_matrix[:, j] = col / norm if norm != 0 else col
    return norm_matrix

# MULTIMOORA METHOD
def multimoora(matrix, weights, criteria_types, sites, criteria):
    n, m = matrix.shape

    # 1. Ratio System (vector normalization, then weighted sum)
    norm_matrix = vector_normalize(matrix)
    for j in range(m):
        if criteria_types[j] == 'min':
            norm_matrix[:, j] = -norm_matrix[:, j]
    ratio_scores = np.dot(norm_matrix, weights)

    # 2. Reference Point Approach (use the same normalization)
    ref_point = np.max(norm_matrix, axis=0)
    # For cost criteria, reference point is min (but we already multiplied by -1)
    ref_distances = np.max(np.abs(norm_matrix - ref_point), axis=1)

    # 3. Full Multiplicative Form
    mult_form = np.ones(n)
    for i in range(n):
        prod = 1.0
        for j in range(m):
            val = matrix[i, j]
            if criteria_types[j] == 'max':
                prod *= val ** weights[j]
            else:  # 'min'
                # Avoid division by zero
                prod *= (1.0 / val) ** weights[j] if val != 0 else 0
        mult_form[i] = prod

    # Rankings
    ratio_rank = rankdata(-ratio_scores, method='min')
    ref_rank = rankdata(ref_distances, method='min')
    mult_rank = rankdata(-mult_form, method='min')

    # Aggregate (Borda count: sum of ranks, lower is better)
    total_rank = ratio_rank + ref_rank + mult_rank

    df = pd.DataFrame({
        'Ratio Score': ratio_scores,
        'Ratio Rank': ratio_rank,
        'Reference Distance': ref_distances,
        'Reference Rank': ref_rank,
        'Multiplicative Score': mult_form,
        'Multiplicative Rank': mult_rank,
        'Total Rank': total_rank,
        'Final Rank': rankdata(total_rank, method='min')
    }, index=sites).sort_values('Final Rank')

    return df

# RUN MULTIMOORA
result_df = multimoora(matrix, weights, criteria_types, sites, criteria)
print(result_df)


       Ratio Score  Ratio Rank  Reference Distance  Reference Rank  \
Site2     0.472260           1            0.379691               2   
Site6     0.388350           2            0.324319               1   
Site8     0.355343           3            0.445609               4   
Site1     0.257189           6            0.436465               3   
Site3     0.281556           5            0.528099               6   
Site5     0.353057           4            0.576327               8   
Site7     0.191882           7            0.525687               5   
Site4     0.187441           8            0.554624               7   

       Multiplicative Score  Multiplicative Rank  Total Rank  Final Rank  
Site2              0.173995                    1           4           1  
Site6              0.154837                    2           5           2  
Site8              0.132795                    3          10           3  
Site1              0.102961                    6          15         